About: In this notebook, we do an analysis of companies based on the relevant ODI concerns. We do the following analysis:

In [ ]:
construct = 'odi'
split = False

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('xtick', labelsize=16)
matplotlib.rc('ytick', labelsize=16)
#plt.rcParams["font.family"] = "lato"
#plt.rcParams["font.style"] = "bold"

plt.rcParams["font.family"] = "lato"
plt.rcParams["font.weight"] = "bold"
plt.rcParams['text.color'] = 'black'
plt.ticklabel_format(style='sci')

import pickle
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# params
from utils import read_config

config = read_config()

DATAROOT = "../data/"
PLOTROOT = "../plots/ODI/"

pros_cons = ['pros', 'cons']

# params
sent_based = True
pro_or_con = config['TEXTS'][construct]
num_goals = len(config['COUNTS'][construct])
THRESHOLD = config['THRESHOLDS'][construct]
PRESET = config['PRESETS'][construct]
primary_goal_shorthand = config['SHORTHANDS'][construct]
final_goals = config['FINAL'][construct]
scoring = config['SCORING'][construct]
reference_text = config['REFERENCES'][construct]
sim_num = 0

In [ ]:
num_goals, pro_or_con, construct

In [ ]:
from utils import sample_data, subset_by_percentile, subset_by_percentile_or_preset

In [ ]:
main_data = pd.read_csv(DATAROOT + "reviews_us_master.csv")

#onek_data = sample_data(main_data, review_threshold = 100)
onek_data = sample_data(main_data, review_threshold = 900)
onek_data = sample_data(main_data)


sustainability = pd.read_csv(DATAROOT + config['REFERENCE_DATAS'][construct], sep = "\t")

main_datas = {}
main_datas_ = {}

for text in pros_cons:
    if sent_based:
        embed_file = "intermediate/%s_%s_sent_embedded.csv" %(construct, text) # sentence-based
    else:
        embed_file = "intermediate/%s_embedded_sim_review.csv" %(text) # original full pro or con
        
    main_datas_[text] = pd.read_csv(DATAROOT+embed_file, sep = "\t")
    #main_datas[text] = []
    #main_datas[text].append(subset_by_percentile_or_preset(main_datas_[text], 0.8, simfield = '_sim_1', preset = 0.20))


In [ ]:
embed_file

In [ ]:
def aggregate_sents(data_90, main_data, sim = 'sim_1'):
    subsetted_data = []
    # for a post with a shortlisted sentence, set it's score to be the maximum scoring sentence
    for g in range(num_goals):
        companies = data_90[g]["company_id"].values
        scores = data_90[g]["%d_%s" %(g, sim)].values
        post_scores = {}
        for n, i in enumerate(companies):
            post_scores[i] = -1

        for n, i in enumerate(companies):
            post_scores[i] = max(post_scores[i], scores[n])
    

        post_score = pd.DataFrame(post_scores.items(), columns = ["company_id", "%d_%s" %(g, sim)])
        subsetted_data.append(post_score.merge(main_data, on = "company_id"))
    return subsetted_data

In [ ]:
pros_cons

In [ ]:
#### run once and save


posted_sim = {}

for sim in ['_sim_1']:
    for text in pros_cons:
        for threshold_ in [THRESHOLD]:
            data_90 = subset_by_percentile_or_preset(main_datas_[text], threshold_,
                                                    simfield = sim, preset = PRESET,
                                                    num = num_goals)
            #data_90 = subset_by_percentile(main_datas_[text], threshold_,
            #                                         simfield = sim)
            if sent_based:
                data_90 = aggregate_sents(data_90, main_data)
            top5_by_goals = pd.DataFrame()
            ### also save full list of companies
            company_list = {}
            posts = []
            for num in range(num_goals):
                posts.append(data_90[num]['company_id'])
            #     print(sustainibility[reference_text][num])
            #     print(data_90[num].groupby("company_name").size().sort_values().tail(5))
                # normalize
                company = pd.DataFrame(data_90[num].groupby("company_name").size(),
                                       columns = ['%d reviews' %num]).reset_index()
                
                # if z-scores are calculated using similiarity
                company['avg sim score'] = list(data_90[num].groupby("company_name").sum()['%d_sim_1' %num].values)
                company_total_reviews = pd.DataFrame(onek_data.groupby("company_name").size(), columns = ['total']).reset_index()
                company = company.merge(company_total_reviews, on = 'company_name')
                company['%d reviews normalized' %(num)] = company['%d reviews' %(num)] / company['total']
                company['avg sim score'] = company['avg sim score'] / company['total']
                top5 = company.sort_values(['%d reviews normalized' %(num)], ascending = False)
                top5[reference_text] = [sustainability[reference_text][num]] * len(top5)
                top5.columns = ['company_name', 'shortlisted reviews', 'avg sim score',
                                'total reviews', 'shortlisted prop', reference_text]
    #             print()
    #             print("normalized: ")
    #             print(top5.head(10))
    #             print()
    #             print("------------------------------------------------------------------------------------------")
                top5_by_goals = top5_by_goals.append(top5, ignore_index=True)
                company_list[sustainability[reference_text][num]] = list(top5['company_name'].values)
    
            posted_sim[sim] = posts
            top5_by_goals.to_csv(DATAROOT + 'intermediate/%s%s_%0.2f_company_ranking.csv' %(text,
                                                                                            sim,
                                                                                            threshold_),
                                 sep = '\t')
            with open(DATAROOT +'intermediate/%s%s_%0.2f_company_ranking.pickle' %(text, sim, threshold_), 'wb') as handle:
                pickle.dump(company_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
              

In [ ]:
with open(DATAROOT +'intermediate/%s%s_%0.2f_company_ranking.pickle' %("cons",
                                                                            sim, THRESHOLD), 'rb') as handle:
    cons = pickle.load(handle)
    
with open(DATAROOT +'intermediate/%s%s_%0.2f_company_ranking.pickle' %("pros",
                                                                            sim, THRESHOLD), 'rb') as handle:
    pros = pickle.load(handle)    
    
cons_table = pd.read_csv(DATAROOT + 'intermediate/%s%s_%0.2f_company_ranking.csv' %("cons",
                                                                                         sim,
                                                                                         THRESHOLD), sep = '\t')
pros_table = pd.read_csv(DATAROOT + 'intermediate/%s%s_%0.2f_company_ranking.csv' %("pros",
                                                                                         sim,
                                                                                         THRESHOLD), sep = '\t')

In [ ]:
pros_table[reference_text].values[0]

In [ ]:
pros_table[pros_table[reference_text] == 'gender equality'].sort_values('shortlisted prop', ascending = False).head(25)

In [ ]:
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows=2, ncols=5, figsize = (15, 10), sharex = True, sharey = 'row')

goals = list(cons.keys())
corrs = []

num = 0

for i in range(4):
    for j in range(5):
        rank1 = cons[goals[final_goals[num]]]
        rank2 = pros[goals[final_goals[num]]]
        #print(goal)
        #print(rank1[:5], rank2[:5])
        
        
        list1 = rank1
        list2 = rank2

        common = list(set(list1).intersection(set(list2)))
        

        plotlist1 = [list1.index(i) for i in common]
        plotlist2 = [list2.index(i) for i in common]
        
        corrs.append(stats.spearmanr(plotlist1, plotlist2)[0])
#         print(sustainability[reference_text][num])
#         print(stats.spearmanr(plotlist1, plotlist2))
        sns.regplot(plotlist1, plotlist2, ax = axs[i][j])
        
        if i == 3:
            axs[i][j].set_xlabel("con rank", fontsize = 14)
        else:
            axs[i][j].set_xlabel("", fontsize = 14)
            
        if j == 0:
            axs[i][j].set_ylabel("pro rank", fontsize = 14)
        else:
            axs[i][j].set_ylabel("", fontsize = 14)
        
        #axs[i][j].set_title("Number of reviews over time (till May 2020)", fontsize = 10)
        subtitle = primary_goal_shorthand[sustainability[reference_text][final_goals[num]]]
        if len(subtitle.split(' ')) > 3:
            midway = len(subtitle) // 2
            subtitle = subtitle[:midway] + "\n-" + subtitle[midway:] 
            
        axs[i][j].set_title("%s (corr = %0.2f)" %(subtitle, corrs[-1]), fontsize = 10)
        #print(stats.spearmanr(rank1, rank2))
        #print()
        num += 1
        if num == len(final_goals):
            break
    if num == len(final_goals):
            break

goals_ = [sustainability[reference_text][i] for i in final_goals]            
            
corr_table = pd.DataFrame({reference_text : goals_,
                      'cor' : corrs})

title = "pros_vs_cons (Def = D%d, threshold = %0.2f, preset = %0.2f)" %(sim_num, THRESHOLD, PRESET)
plt.suptitle(title, fontsize = 20)
plt.savefig(PLOTROOT + "companies/%s.pdf" %(title))
#plt.tight_layout()

#corr_table.round(3)

In [ ]:
corr_table.mean()

In [ ]:
from utils import aggregate_area

In [ ]:
scoring

In [ ]:
company_data = {}

for text in [text]:
    data_90 = subset_by_percentile_or_preset(main_datas_[text], THRESHOLD, preset = PRESET, num = num_goals)
    if sent_based:
        data_90 = aggregate_sents(data_90, main_data)              
    company_data[text] = aggregate_area(data_90, final_goals, onek_data,
                                      sustainability,
                                      construct = construct,
                                      #metric = 'pro proportion',
                                      area_name = 'company_name',
                                      scaled = True, scoring = scoring,
                                      find_pcs = False,
                                      reference_text = reference_text)

In [ ]:
company_data[text][1].columns

In [ ]:
company_data[text][1].corr()

In [ ]:
cross_corr_table = company_data[text][1].copy()[['company_name', '1 avg sim score', '2 avg sim score',
                                               '3 avg sim score', '5 avg sim score', 'composite']]
cross_corr_table.columns = ['company_name', 'Depressed mood', 'Sleep alterations', 'Fatigue / loss of energy',
       'Feelings of worthlessness', 'composite']
cross_corr_table.corr()

In [ ]:
df = company_data[text][1].copy()

cross_corr_vars = ['ODI',
                   'Urban Population'] + ['Depression',
                                          'Wealth',
                                          'Creativity'] + ['O', 'C', 'E', 'A', 'N']
                                                               
                                    

rows = []
for i in range(0, len(cross_corr_vars)):
    var1 = cross_corr_vars[i]
    row = []
    for j in range(0, i+1):
        var2 = cross_corr_vars[j]
        corr, p = stats.pearsonr(df[var1], df[var2])
        if var1 == var2:
            row.append("---")
            continue
        if p < 0.005:
            stars = "***"
        elif p < 0.01:
            stars = "**"
        elif p < 0.05:
            stars = "*"
        else:
            stars = ""
        row.append(str("%0.3f" %corr)+stars)
    rows.append(row)
    
cross_corr = pd.DataFrame(rows, columns = cross_corr_vars)    
cross_corr["var"] = cross_corr_vars
cross_corr = cross_corr.set_index("var")
cross_corr = cross_corr.fillna(" ")
cross_corr

In [ ]:
print(cross_corr_table.corr().round(3).to_latex())

In [ ]:
company_data[text][1]

In [ ]:
company_data[text][1].corr()

In [ ]:
company_data[text][1]

In [ ]:
stocks = pd.read_csv(DATAROOT+"companies_stock.csv")

In [ ]:
stocks.head()

## Actual Company analysis

We fix the threshold here.

In [ ]:
industry_data = main_data[['company_name', 'GICSSector', 'GICSSubIndustry']]
industry_data = industry_data.drop_duplicates('company_name', keep = "first")
industry_data

In [ ]:
industry_data = industry_data.fillna('Missing')
industry_data.groupby('GICSSector').size().sort_values().plot(kind = 'bar')

In [ ]:
table = company_data[pro_or_con][1].merge(industry_data, on = 'company_name')

In [ ]:
table.columns

In [ ]:
submetric = 'avg sim score'

In [ ]:
metric = '1 %s' %submetric
ax = table.sort_values(metric,
                                    ascending = False).head(10)[['company_name',
                                                                 'GICSSector']].groupby('GICSSector').size().sort_values().plot(kind = 'bar')
ax.set_title("Top 10 companies for\n%s\n(%s)" %(sustainability[reference_text][i], metric), fontsize = 20)
ax.set_ylabel("# companies", fontsize = 18)
ax.set_xlabel('industry', fontsize = 18)

In [ ]:
metric = '3 %s' %submetric
ax = table.sort_values(metric, ascending = False).tail(10)[['company_name', 'GICSSector']]\
        .groupby('GICSSector').size().sort_values().plot(kind = 'bar')
ax.set_title("Low ranked companies for\n%s\n(%s)" %(sustainability[reference_text][i], metric), fontsize = 20)
ax.set_ylabel("# companies", fontsize = 18)
ax.set_xlabel('industry', fontsize = 18)

## sustainability score by sector

In [ ]:
# remove missing and take those industries which have at least 5 or more companies in our dataset
industries = ['Consumer Discretionary', 'Information Technology',
              'Health Care', 'Financials', 'Industrials', 'Consumer Staples']

In [ ]:
sector = 'GICSSector'
industry_data = table[table[sector].isin(industries)]
industry_data.groupby(sector).mean()

In [ ]:
sns.boxplot(data = industry_data, x = 'composite', y = 'GICSSector')

In [ ]:
def scale_var(data, var):
    scaled_data = [i[0]*100 for i in list(scaler.fit_transform(data[[var]]))]
    return scaled_data

def standardize(data, var):
    return (data[var] - data[var].mean())/data[var].std(ddof=0)

In [ ]:
# scale composite and welfare
industry_data['composite'] = scale_var(industry_data, 'composite')
if construct == 'odi':
    for col in ['physiological', 'psychological']:
        industry_data[col] = scale_var(industry_data, col)
# industry_data['PCA1'] = scale_var(industry_data, 'PCA1')
# industry_data['rank-diff'] = scale_var(industry_data, 'rank-diff')
# industry_data['PCA2'] = scale_var(industry_data, 'PCA2')

# z-score and scale the others
for goal in final_goals:
    industry_data['%d avg sim score' %goal] = standardize(industry_data, '%d avg sim score' %goal)
    industry_data['%d avg sim score' %goal] = scale_var(industry_data, '%d avg sim score' %goal)

In [ ]:
value_vars=['%d avg sim score' %i for i in final_goals]
value_vars.extend(['composite'])
if construct == 'odi':
    value_vars.extend(['physiological', 'psychological'])
industry_data_melted = pd.melt(industry_data, id_vars=['company_name', 'GICSSector'],
                               value_vars = value_vars)
industry_data_melted

In [ ]:
new_names = dict([("%d avg sim score" %i, primary_goal_shorthand[sustainability[reference_text][i]]) for i in final_goals])
new_names['composite'] = 'composite'
if construct == 'odi':
    new_names['physiological'] = 'physiological'
    new_names['psychological'] = 'psychological'

# new_names['PCA1'] = composite
# new_names['PCA2'] = 'financial'
# new_names['rank-diff'] = 'welfare_premium'

In [ ]:
industry_data_melted['variable'] = industry_data_melted['variable'].map(new_names)
industry_data_melted

In [ ]:
industry_data_melted['variable'].unique()

In [ ]:
#sns.set(font="Lato")

In [ ]:
col_order = ['composite']#list(new_names.values())
if split:
    col_order.extend(['psychological', 'physiological'])
    col_wrap = 3
else:
    col_wrap = 1
#[
#             composite, 'financial', 'welfare_premium',
#              'health', 'education', 'gender equality', 'financial benefits',
#        'support infrastructure', 'supportive environment'
#            ]
g = sns.FacetGrid(industry_data_melted, col="variable", height = 3, #width = 10,
                  col_wrap = col_wrap, sharex = False,
                  col_order = col_order)
g.map_dataframe(sns.boxplot, x = "value", y = "GICSSector")
g.set_axis_labels("ODI Score", "Industry Sector", fontsize = 18)
# g.set_titles(col_template="{col_name}", size = 18)
g.set_titles("", size = 18)
g.tight_layout()
g.add_legend()
g.savefig(PLOTROOT + "companies/sector_vs_sustainability.pdf")

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (7, 3.5))
ax = sns.boxplot(data = industry_data, x = 'composite', y = 'GICSSector', color = 'steelblue')
ax.set_xlabel('ODI score', fontweight = 'bold', fontsize = 20)
ax.set_ylabel('Industry Sector', fontweight = 'bold', fontsize = 20)
plt.tight_layout()
plt.savefig(PLOTROOT + "companies/sector_vs_sustainability.pdf")

In [ ]:
# col_order = list(industry_data_melted['variable'].unique())[:-3]
# col_order

In [ ]:
# g = sns.FacetGrid(industry_data_melted, col="variable", height = 6, col_wrap = 3, sharex = False, col_order = col_order)
# g.map_dataframe(sns.boxplot, x = "value", y = "GICSSector")
# g.set_axis_labels("Score", "Sector", fontsize = 18)
# g.set_titles(col_template="{col_name}", size = 18)
# g.tight_layout()
# g.add_legend()
# #g.savefig(PLOTROOT + "companies/sector_vs_sustainability_ALL.pdf")

In [ ]:
# sns.distplot(industry_data['composite'], label = 'composite')
# sns.distplot(industry_data['rank-diff'], label = 'emergent')
# sns.distplot(industry_data['2 avg sim score'], label = 'health')
# plt.legend()

In [ ]:
# # do some ANCOVA --- test if the different is S(c, g) and S(c) are significant while controlling # reviews
# from scipy.stats import f_oneway

In [ ]:
industry_data[['composite', 'GICSSector']].head()

In [ ]:
df = industry_data_melted.copy()

In [ ]:
dfs = [industry_data.groupby('GICSSector').get_group(x)[['composite']] for x in industry_data.groupby('GICSSector').groups]

In [ ]:
len(dfs)

In [ ]:
fvalue, pvalue = stats.f_oneway(dfs[0], dfs[1], dfs[2], dfs[3], dfs[4], dfs[5])
print(fvalue, pvalue)

In [ ]:
def find_anova(var = 'composite'):
    data = industry_data_melted[industry_data_melted['variable'] == var]
    anova_lists = []
    for industry in set(data['GICSSector'].unique()):
        data_subset = data[data['GICSSector'] == industry]
        anova_lists.append(list(data_subset['value'].values))
    return f_oneway(anova_lists[0], anova_lists[1], anova_lists[2],
         anova_lists[3], anova_lists[4], anova_lists[5])

In [ ]:
# https://www.marsja.se/python-manova-made-easy-using-statsmodels/
from statsmodels.multivariate.manova import MANOVA

industry_data_manova = industry_data.copy()
industry_data_manova.columns = ['company_name', 'health', 'education', 'gender',
       'pay', 'infra', 'peace',
       'composite', 'PCA1', 'PCA2', 'pca-one-rank', 'pca-two-rank',
       'rank-diff', 'GICSSector', 'GICSSubIndustry']

maov = MANOVA.from_formula('health + education + \
                            gender + pay + infra + peace + composite ~ GICSSector', data = industry_data_manova) 

print(maov.mv_test())

### Repeat rating correlation analysis

load main data, get aggregate rating for each company, get 2019 data and then correlate

In [ ]:
rating_cols = [i for i in main_data.columns if 'rating' in i]
rating_cols

In [ ]:
main_data.isna().sum()

In [ ]:
onek_data_2019_company = onek_data.groupby('company_name').mean()[rating_cols]
onek_data_2019_company = onek_data_2019_company.reset_index()
onek_data_2019_company.columns = ['company_name', 'rating_balance',
 'rating_career',
 'rating_comp',
 'rating_culture',
 'rating_mgmt',
 'rating_overall']
onek_data_2019_company

In [ ]:
len(onek_data_2019_company), onek_data_2019_company.isna().sum()

In [ ]:
onek_data_2019_company = onek_data_2019_company.dropna(subset = ['rating_mgmt'])

In [ ]:
for rating in ['rating_culture', 'rating_balance', 'rating_comp', 'rating_mgmt', 'rating_career']:
    print(rating)
    for num in final_goals:
        data_ = company_data[pro_or_con][1].merge(onek_data_2019_company, on = "company_name")
        print(sustainability[reference_text][num], stats.pearsonr(data_['%d %s' %(num, submetric)],
                                                      data_[rating]))
    print()    

In [ ]:
onek_data_2019_company_reviews = onek_data.groupby('company_name').size().reset_index()
onek_data_2019_company_reviews.columns = ['company_name', 'reviews']
onek_data_2019_company = onek_data_2019_company.merge(onek_data_2019_company_reviews, on = 'company_name')
onek_data_2019_company['total reviews (logged)'] = np.log(onek_data_2019_company['reviews'])
onek_data_2019_company.head()

In [ ]:
onek_data_2019_company['rating_avg'] = onek_data_2019_company[rating_cols].mean(axis = 1)
onek_data_2019_company.head()

In [ ]:
table = []
metric = 'avg sim score'
for rating in ['rating_culture', 'rating_balance', 'rating_comp',
               'rating_mgmt', 'rating_career', 'rating_overall',
              'total reviews (logged)'
              ]:
    #print(rating)
    row = [rating]
    for num in final_goals:
        data_ = company_data[pro_or_con][1].merge(onek_data_2019_company, on = "company_name")
        #print(sustainability[reference_text][num], stats.pearsonr(data_['pro proportion'],
        #                                              data_[rating]))
        corr, p_val = stats.pearsonr(data_["%d %s" %(num, metric)], data_[rating])
        if p_val < 0.005:
            sig = "***"
        elif p_val < 0.01:
            sig = "**"
        elif p_val < 0.05:
            sig = "*"
        else:
            sig = ''
        row.append('%0.2f%s'%(corr, sig))
        #row.append(p_val)
    #data_ = onek_data_2019_company_reviews.merge(onek_data_2019_company, on = "company")
    corr, p_val = stats.pearsonr(np.log(data_['reviews']), data_[rating])
    if p_val < 0.005:
        sig = "***"
    elif p_val < 0.01:
        sig = "**"
    elif p_val < 0.05:
        sig = "*"
    else:
        sig = ''
    row.append('%0.2f%s'%(corr, sig))
#     corr, p_val = stats.pearsonr(data_['rating_avg'], data_[rating])
#     row.append(corr)
    table.append(row)
    #print()   

columns = ['rating']    
columns.extend([primary_goal_shorthand[sustainability[reference_text][i]] for i in final_goals])
columns.append('total reviews (logged)')
#columns.append('avg rating')
rating_correlation_table = pd.DataFrame(table, columns = columns)    

In [ ]:
rating_correlation_table['rating'] = ['culture', 'balance', 'company', 'management', 'career', 'overall', 'total reviews (logged)']
rating_correlation_table

In [ ]:
PRESET, THRESHOLD, scoring

In [ ]:
#rating_correlation_table.to_csv("../results/rating_correlation_table_weighted.csv", sep = '\t', index = False)

In [ ]:
# summarize by different scoring criteria

def find_rating_correlation(company_data):
    table = []
    metric = 'avg sim score'
    for rating in ['rating_culture', 'rating_balance', 'rating_comp',
               'rating_mgmt', 'rating_career', 'rating_overall',
              'total reviews (logged)'
              ]:
    
        row = [rating]
        for num in final_goals:
            data_ = company_data.merge(onek_data_2019_company, on = "company_name")
            corr, p_val = stats.pearsonr(data_["%d %s" %(num, metric)], data_[rating])
            if p_val < 0.005:
                sig = "***"
            elif p_val < 0.01:
                sig = "**"
            elif p_val < 0.05:
                sig = "*"
            else:
                sig = ''
            row.append('%0.2f%s'%(corr, sig))
        corr, p_val = stats.pearsonr(np.log(data_['reviews']), data_[rating])
        if p_val < 0.005:
            sig = "***"
        elif p_val < 0.01:
            sig = "**"
        elif p_val < 0.05:
            sig = "*"
        else:
            sig = ''
        row.append('%0.2f%s'%(corr, sig))
        table.append(row)

    columns = ['rating']    
    columns.extend([primary_goal_shorthand[sustainability[reference_text][i]] for i in final_goals])
    columns.append('total reviews (logged)')
    rating_correlation_table = pd.DataFrame(table, columns = columns)  
    rating_correlation_table['rating'] = ['culture', 'balance', 'company', 'management', 'career', 'overall', 'total reviews (logged)']
    
    return rating_correlation_table


In [ ]:
rating_tables = {}

for scoring in ['log', 'exp', 'normal']:
    data = aggregate_area(data_90, final_goals, onek_data,
                                      sustainability,
                                      #metric = 'pro proportion',
                                      construct = construct,
                                      area_name = 'company_name',
                                      scaled = False, scoring = scoring,
                                      find_pcs = False,
                                      reference_text = reference_text)
    rating_tables[scoring] = find_rating_correlation(data[1])
    

In [ ]:
rating_tables['normal']

In [ ]:
rating_tables['log']

In [ ]:
rating_tables['exp']

### correlation between overall score and other goal scores

In [ ]:
sustainable_score_corr = company_data[pro_or_con][1].rename(new_names, axis=1)
#ax = sustainable_score_corr.corr()['composite'].sort_values().plot(kind = 'bar')

In [ ]:
new_names

In [ ]:
len(company_data[pro_or_con][1]), len(industry_data)

Based on the rating correlation results, we will use the "bored" and "never ending dimensions", and possibly revisist "distracted" later.

## company binning by percentile

In [ ]:
stocks = pd.read_csv(DATAROOT+"companies_stock.csv")
stocks = stocks.rename({'company' : 'company_name'},  axis=1)
stocks

In [ ]:
stocks.isna().sum()

In [ ]:
from scipy import stats
import math

def plot_gm_bins(company_table, metric = '2 avg sim score', start_year = '2009', end_year = '2019'):    
    gms = []
    gses = []
    for percentile in range(30, 40):
        data = company_table.copy()
        data = data.dropna(subset = ['stock_2009', 'stock_2019'])
        data['total_stock_growth'] = data['stock_2019'] / data['stock_2009']
        data['log_total_stock_growth'] = np.log(data['stock_2019'] / data['stock_2009'])
        threshold = data[metric].quantile(percentile * 0.025)
        percentile_companies = data[data[metric] > threshold]
    
        gm = stats.gmean(percentile_companies['total_stock_growth'])
        gms.append(gm)
        
        #gse = stats.gstd(percentile_companies['total_stock_growth'])
        num = gm / np.sqrt(len(percentile_companies)) *\
            np.std(percentile_companies['log_total_stock_growth'])
        gse = num 
                                        
        gses.append(gse)
    return gms, gses

In [ ]:
goal_gms = {}
goal_gses = {}
for goal in final_goals:
    #data = table_weighted[table_weighted[reference_text] == sustainability[reference_text][goal]]
    data = company_data[pro_or_con][1]
    data = data.merge(stocks, on = "company_name")
    metric = '%d avg sim score' %goal
    goal_gms[goal], goal_gses[goal] = plot_gm_bins(data, metric)

    
data = data.merge(onek_data_2019_company[['company_name', 'reviews']], on = 'company_name')    
goal_gms['# Reviews'], goal_gses['# Reviews'] = plot_gm_bins(data, metric = 'reviews') 
goal_gms['ODI'], goal_gses['ODI'] = plot_gm_bins(data, metric = 'composite') 
# goal_gms['rank-diff'], goal_gses['rank-diff'] = plot_gm_bins(data, metric = 'rank-diff') 
# goal_gms['PCA2'], goal_gses['PCA2'] = plot_gm_bins(data, metric = 'PCA2') 

if construct == 'odi':
    goal_gms['physiological'], goal_gses['physiological'] = plot_gm_bins(data, metric = 'physiological') 
    goal_gms['psychological'], goal_gses['psychological'] = plot_gm_bins(data, metric = 'psychological') 


In [ ]:
goal_gms_table = pd.DataFrame(goal_gms)
goal_gses_table = pd.DataFrame(goal_gses)

extras = ['# Reviews', 'ODI']
if construct == 'odi':
    extras.extend(['psychological', 'physiological'])

goal_gms_table.columns = final_goals + extras
goal_gses_table.columns = ["{} SE".format(i) for i in goal_gms_table.columns]
goal_gms_table = pd.concat([goal_gms_table, goal_gses_table], axis=1)
goal_gms_table['percentile'] = [(1-i*0.025)*100 for i in range(30, 40)]

In [ ]:
goal_gms_table['percentile']

In [ ]:
goal_gms_table = goal_gms_table.reindex(index = goal_gms_table.index[::-1])


In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (18, 5))
factor = 0
for goal in final_goals:
#     ax = sns.lineplot(data = goal_gms_table, x = "percentile",
#                       y = goal, label = goal_shorthand[sustainability[reference_text][goal]])
    
    if type(goal) != str:
        goal_gms_table['percentile %d' %goal] = goal_gms_table['percentile'] + factor
        #goal_gms_table['percentile %d' %goal] = [str('%0.2f' %i) for i in goal_gms_table['percentile %d' %goal]]
        goal_gms_table.plot(x = "percentile %d" %goal, y = goal, yerr = "%d SE" %goal, capsize=4,
                        label = primary_goal_shorthand[sustainability[reference_text][goal]], ax = ax)
    else:
        goal_gms_table['percentile %s' %goal] = goal_gms_table['percentile'] + factor
        goal_gms_table.plot(x = 'percentile %s' %goal , y = goal, yerr = "%s SE" %goal, capsize=4,
                        label = goal, ax = ax)
    factor += 0.01
    
ax.invert_xaxis()    
ax.set_ylabel('GM Stock Growth (2009 - 2019)', fontsize = 16)
ax.set_xlabel('Sustainability score percentile', fontsize = 16)
plt.legend(bbox_to_anchor=(1, 1), fontsize = 16)
plt.tight_layout()
#plt.savefig(PLOTROOT + 'companies/geometric_mean_bin_plot_gender_pay_reviews_composite.pdf')
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (10, 5))
plt.locator_params(axis='y', nbins=6)

factor = 0

goals = ['ODI','# Reviews']
if split:
    goals.extend(['physiological', 'psychological'])

for goal in goals:
#     ax = sns.lineplot(data = goal_gms_table, x = "percentile",
#                       y = goal, label = goal_shorthand[sustainability[reference_text][goal]])
    
    if type(goal) != str:
        goal_gms_table['percentile %d' %goal] = goal_gms_table['percentile'] + factor
        #goal_gms_table['percentile %d' %goal] = [str('%0.2f' %i) for i in goal_gms_table['percentile %d' %goal]]
        goal_gms_table.plot(x = "percentile %d" %goal, y = goal, yerr = "%d SE" %goal, capsize=4,
                        label = primary_goal_shorthand[sustainability[reference_text][goal]], ax = ax)
    else:
        goal_gms_table['percentile %s' %goal] = goal_gms_table['percentile'] + factor
        if goal == 'composite':
            goal_gms_table.plot(x = 'percentile %s' %goal , y = goal, yerr = "%s SE" %goal, capsize=4,
                        label = "composite score", ax = ax)
        elif goal == '# reviews':
            goal_gms_table.plot(x = 'percentile %s' %goal , y = goal, yerr = "%s SE" %goal, capsize=4,
                        label = goal, ax = ax, style = '.', color = 'grey', alpha = 0.5)
        else:
            goal_gms_table.plot(x = 'percentile %s' %goal , y = goal, yerr = "%s SE" %goal, capsize=4,
                        label = goal, ax = ax)
            
    factor -= 0.3
    
ax.invert_xaxis()    
ax.set_ylabel('GM', fontsize = 34, fontweight = 'bold')
ax.set_xlabel('Score of top X% of companies', fontsize = 24, fontweight = 'bold')
ax.tick_params(axis = "x", labelsize = 16) 
ax.set_yticklabels(ax.get_yticks(), size = 16)
#ax.set_xticklabels(ax.get_xticks(), size = 16)
ax.set_yticklabels(ax.get_yticks().astype(int))
ax.set_title("Geometric Mean (GM) of Stock Growth of Companies", fontsize = 26, fontweight = 'bold')
plt.legend(fontsize = 18)
plt.tight_layout()
plt.savefig(PLOTROOT + 'companies/geometric_mean_bin_plot_reviews_composite_welfare_financial_half_finer.pdf')
plt.show()

In [ ]:
goal_gms_table

The following analysis doesn't make sense in the boredom / ODI context, at least not until we define what the PCs represent

In [ ]:
data